In [1]:
# import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from sklearn.linear_model import Lasso
from scipy.io import savemat

import pysindy as ps

def hyperLorenzEqs(t,X,a,b,c,d):
    x = X[0]
    y = X[1]
    z = X[2]
    w = X[3]
    x_dot = -a*x+a*y+w
    y_dot = c*x-y-x*z
    z_dot = -b*z+x*y
    w_dot = d*w-x*z
    return [x_dot,y_dot,z_dot,w_dot]

In [2]:
# integration keywords for solve_ivp, typically needed for chaotic systems
integrator_keywords = {}
integrator_keywords["rtol"] = 1e-12
integrator_keywords["method"] = "LSODA"
integrator_keywords["atol"] = 1e-12

In [3]:
# generate clean data
dt = 0.001
t_train = np.arange(0, 10, dt)
t_train_span = (t_train[0], t_train[-1])
X0 = [5,8,12,21]

a = 10
b = 2.667
c = 28
d = 1.1

X_clean = solve_ivp(hyperLorenzEqs,t_train_span,X0,args=[a,b,c,d],method="RK45",t_eval=t_train,).y.T

# true coefficient matrix 
XD = ps.FiniteDifference()._differentiate(X_clean,t=dt)
poly_lib = ps.PolynomialLibrary(degree=2,include_bias=True)
model = ps.SINDy(feature_library=poly_lib)
model.fit(X_clean,x_dot=XD,t=dt)
w_true = model.coefficients().T
w_true[0,1] = 0
w_true[0,2] = 0

# noisy trajectories
MD = 10*1000*4
arr_sig_NR = np.hstack((np.logspace(-6,-2,5),np.linspace(0.05,1,15)))   # array of noise ratios
N_noise = 20   # number of noises to try at each noise level
arr_X_noisy = np.zeros((X_clean.shape[0],X_clean.shape[1],len(arr_sig_NR),N_noise)) # allocte a 4-way array for storing noisy trajectory data

rng = np.random.default_rng(seed=0)
for i in range(0,len(arr_sig_NR)):
    sig_NR = arr_sig_NR[i]
    sig = sig_NR*np.linalg.norm(X_clean,"fro")/np.sqrt(MD)
    for j in range(0,N_noise):
        X_noisy = X_clean+rng.normal(0,sig,size=X_clean.shape)
        arr_X_noisy[:,:,i,j] = X_noisy

# SINDy
arr_w_SINDy = np.zeros((w_true.shape[0],w_true.shape[1],len(arr_sig_NR),N_noise))
for i in range(0,len(arr_sig_NR)):
    for j in range(0,N_noise):
        X_noisy = arr_X_noisy[:,:,i,j]
        XD = ps.FiniteDifference()._differentiate(X_noisy,t=dt)
        
        model = ps.SINDy(feature_library=poly_lib)
        model.fit(X_noisy,x_dot=XD,t=dt)
        w_ident = model.coefficients().T

        arr_w_SINDy[:,:,i,j] = w_ident
        print(f"Progress 1/2: {i*N_noise+(j+1)}/{len(arr_sig_NR)*N_noise}")

# weak SINDy
ode_lib = ps.WeakPDELibrary(
    function_library=ps.PolynomialLibrary(degree=2,include_bias=True),
    spatiotemporal_grid=t_train,
    is_uniform=True,
    K=1000,
)
optimizer = ps.SR3(
    reg_weight_lam=0.3, 
    regularizer="L0", 
    max_iter=1000, 
    normalize_columns=False, 
    tol=1e-1
)

arr_w_WSINDy = np.zeros((w_true.shape[0],w_true.shape[1],len(arr_sig_NR),N_noise))
for i in range(0,len(arr_sig_NR)):
    for j in range(0,N_noise):
        X_noisy = arr_X_noisy[:,:,i,j]
        
        model = ps.SINDy(feature_library=ode_lib, optimizer=optimizer)
        model.fit(X_noisy,t_train)
        w_ident = model.coefficients().T

        arr_w_WSINDy[:,:,i,j] = w_ident
        print(f"Progress 2/2: {i*N_noise+(j+1)}/{len(arr_sig_NR)*N_noise}")

Progress 1/2: 1/400
Progress 1/2: 2/400
Progress 1/2: 3/400
Progress 1/2: 4/400
Progress 1/2: 5/400
Progress 1/2: 6/400
Progress 1/2: 7/400
Progress 1/2: 8/400
Progress 1/2: 9/400
Progress 1/2: 10/400
Progress 1/2: 11/400
Progress 1/2: 12/400
Progress 1/2: 13/400
Progress 1/2: 14/400
Progress 1/2: 15/400
Progress 1/2: 16/400
Progress 1/2: 17/400
Progress 1/2: 18/400
Progress 1/2: 19/400
Progress 1/2: 20/400
Progress 1/2: 21/400
Progress 1/2: 22/400
Progress 1/2: 23/400
Progress 1/2: 24/400
Progress 1/2: 25/400
Progress 1/2: 26/400
Progress 1/2: 27/400
Progress 1/2: 28/400
Progress 1/2: 29/400
Progress 1/2: 30/400
Progress 1/2: 31/400
Progress 1/2: 32/400
Progress 1/2: 33/400
Progress 1/2: 34/400
Progress 1/2: 35/400
Progress 1/2: 36/400
Progress 1/2: 37/400
Progress 1/2: 38/400
Progress 1/2: 39/400
Progress 1/2: 40/400
Progress 1/2: 41/400
Progress 1/2: 42/400
Progress 1/2: 43/400
Progress 1/2: 44/400
Progress 1/2: 45/400
Progress 1/2: 46/400
Progress 1/2: 47/400
Progress 1/2: 48/400
P

d:\anaconda3\envs\cleanfish\lib\site-packages\pysindy\feature_library\weak_pde_library.py:177: UserWarning: is_uniform and periodic have been deprecated.in favor of differetiation_method and diff_kwargs.
  warnings.warn(
d:\anaconda3\envs\cleanfish\lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(


Progress 2/2: 1/400


d:\anaconda3\envs\cleanfish\lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(


Progress 2/2: 2/400


...

d:\anaconda3\envs\cleanfish\lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(


Progress 2/2: 398/400


d:\anaconda3\envs\cleanfish\lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(


Progress 2/2: 399/400
Progress 2/2: 400/400


d:\anaconda3\envs\cleanfish\lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(


In [4]:
data = dict()
data["w_true"] = w_true
data["arr_sig_NR"] = arr_sig_NR
data["arr_w_SINDy"] = arr_w_SINDy
data["arr_w_WSINDy"] = arr_w_WSINDy

savemat("hyperLorenz_poly2_ps.mat",data)